In [ ]:
# -*- coding: utf-8 -*-

import numpy as np
import argparse
import time
import cv2
import os
import paho.mqtt.client as paho
import json

# Official Cloudmqtt Code don't touch
def on_connect(client, userdata, flags, rc):
    print("Connection returned result: " + str(rc) )
def on_message(client, userdata, msg):
    print("topic: "+msg.topic)
    print("payload: "+str(msg.payload))
def on_publish(client, obj, mid):
    print("mid: " + str(mid))
def on_subscribe(client, obj, mid, granted_qos):
    print("Subscribed: " + str(mid) + " " + str(granted_qos))
    
# Set all Yolo files

Sens =0.5  # Value 1 > Sens > 0 (This is the minimum prediction value accepted)
labelsPath = "obj.names"
weightsPath = "yolov3_last.weights"
configPath = "yolov3.cfg"

# Setting All MQTT Callback Functions

mqtt = paho.Client()
mqtt.on_connect = on_connect
mqtt.on_message = on_message
mqtt.on_publish = on_publish
mqtt.on_subscribe = on_subscribe

# Set MQTT Credentials

mqtt.username_pw_set("USER", "PASSWORD")
mqtt.connect("URL", PORT, keepalive=60)
rc = 0

# Select your Image Source, in this case my camera
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_EXPOSURE,-10)

labels = []

## Dont touch, this is the official algorithm to run yolo with opencv-contrib

while 1: 
    mqtt.loop()
    ret, image = cap.read()
    net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
    LABELS = open(labelsPath).read().strip().split("\n")
    np.random.seed(42)
    COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")
    cv2.imwrite('image.jpg', image)
    image = cv2.imread("out.jpg")
    (H, W) = image.shape[:2]
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
     swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward(ln)
    end = time.time()
    boxes = []
    confidences = []
    classIDs = []
    for output in layerOutputs:
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > Sens:
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.5,0.3)
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            height, width, channels = image.shape
            text = "{}".format(LABELS[classIDs[i]])
            labels.append(LABELS[classIDs[i]])
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
                0.5, color, 2)
                
    # Saving the processed image
    cv2.imwrite('process.jpg', image)
    
    # Counting how many Ripe and Mature tomatoes
    x =  '{ "Ripe":0, "Mature":0}'
    y = json.loads(x)
    
    for x in range(len(labels)):
        if(labels[x]=="Ripe"):
            y["Ripe"]=y["Ripe"]+1
        else:
            y["Mature"]=y["Mature"]+1
    
    # Sending the value with MQTT
            
    mqtt.publish('/AgroJetson', json.dumps(y))
    labels = []
    
    # Sleep 30 minutes
    
    time.sleep(60 * 30) # 60 Seconds * 30 = 30 Minutes

    
cap.release()
cv2.destroyAllWindows()